In [58]:
import pandas as pd

df = pd.read_csv('/root/KRX_LLM/data/mcqa_data1.csv')
def is_numeric(val):
    try:
        int(val)
        return True
    except ValueError:
        return False

print(len(df))
df = df[df['answer'].apply(is_numeric)]
df = df.reset_index(drop=True)

df['question'] = df['question'].map(lambda x : x)
print(len(df))

625
622


In [59]:
df.head(10)

,question,answer
0,"['AI 시스템이 금융 서비스에서 의사결정을 내릴 때, 윤리적 고려사항과 투명성을 ...",1
1,"['은행에서 고객 맞춤형 서비스 제공을 위해 어떤 기술의 도입이 필수적이며, 이와 ...",1
2,['AI를 사용하여 보험사가 고객 경험을 향상시키는 방법과 이러한 향상이 고객에게 ...,1
3,['어떤 기술이나 방법이 금융 서비스 접근성을 높이는 데 가장 적합한지에 대해 다음...,3
4,"['AI 기반 신용 위험 평가 시스템의 장점을 고려할 때, 마이크로파이낸스 기관에서...",3
5,"['파생상품과 알고리즘 거래는 긴밀하게 연결되어 있으며, 각기 다른 시장 서비스 제...",3
6,['2019년에 발표된 AI 시스템에 대한 가이드라인에서 강조된 요소 중에서 금융 ...,5
7,"['AI 기반 거래 전략과 알고리즘 거래 전략 간의 차이를 고려할 때, AI 전략이...",1
8,['AI 기반 금융 서비스에서 고객의 투자 포트폴리오 최적화를 위한 기술의 주요 기...,2
9,['AI 시스템이 금융 분야에서 고객 서비스와 포트폴리오 관리의 효과성 증대가 예상...,1


In [60]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), 'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_TEAM_API_KEY')
client = openai.OpenAI(api_key=openai_key)

# 중복 정답 유무 체크

In [61]:
system_prompt ="""
You are a financial expert.
Read the following questions and choose the answer.
It could be multiple correct answers.
Just write a number or numbers.
"""

user_prompt="""
Question : {}
"""

In [62]:
from tqdm import tqdm

gpt_answer = []
for _, row_item in tqdm(df.iterrows(), total=len(df)):
    answer = int(row_item['answer'])
    question = str(row_item['question'])

    system_msg = {'role': 'system', 'content':system_prompt}
    user_msg = {'role':'user', 'content':user_prompt.format(question)}

    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [system_msg, user_msg]
    )

    response_text = response.choices[0].message.content

    gpt_4o_answer = []
    for i in range(6):
        if str(i) in response_text:
            gpt_4o_answer.append(i)

    gpt_answer.append(gpt_4o_answer)

100%|██████████| 622/622 [06:59<00:00,  1.48it/s]


# 중복 정답 선택지 바꾸기

In [63]:
fixed_system_prompt = """
Your role is to correct the options for financial matters. You will be given the problem and the option number that you will need to rewrite. Please correct the given option number's sentence for your question by meeting the following conditions.
1. The selection must not deviate from the subject of the problem.
2. It should undoubtedly be corrected with a perfect incorrect answer.
3. It must not be modified in the same sense as other fingerprints.
4. The length selected must be similar to the length of other options.
You must only write correct options which satisfy above conditions.
"""

fixed_user_prompt="""
Question: {}
fixed_numbers : {}
"""

In [64]:
fixed_results = []
df['gpt_answer'] = gpt_answer
df

,question,answer,gpt_answer
0,"['AI 시스템이 금융 서비스에서 의사결정을 내릴 때, 윤리적 고려사항과 투명성을 ...",1,"[1, 5]"
1,"['은행에서 고객 맞춤형 서비스 제공을 위해 어떤 기술의 도입이 필수적이며, 이와 ...",1,[1]
2,['AI를 사용하여 보험사가 고객 경험을 향상시키는 방법과 이러한 향상이 고객에게 ...,1,"[1, 5]"
3,['어떤 기술이나 방법이 금융 서비스 접근성을 높이는 데 가장 적합한지에 대해 다음...,3,"[2, 4, 5]"
4,"['AI 기반 신용 위험 평가 시스템의 장점을 고려할 때, 마이크로파이낸스 기관에서...",3,[3]
...,...,...,...
617,"['재정 당국이 금융 기관의 감독 및 집행을 위한 조치를 취할 때, 어떤 접근 방식...",3,[3]
618,['사이프러스의 2013년 은행 위기 대응과 캐나다의 금융 안정성 유지 조치들 간에...,4,[4]
619,['유로존 국가들의 재정적자 문제 해결을 위한 스칼 컴팩트 도입 시 중요한 요소 중...,1,"[1, 3]"
620,['인도와 브라질의 예금 보험 시스템이 각각의 은행의 위험 프로파일에 따라 달라지는...,5,"[1, 3, 5]"


In [42]:


for idx, row_item in tqdm(df.iterrows(), total=len(df)):
    question = row_item['question']
    answer = row_item['answer']
    gpt_answer = row_item['gpt_answer']
    fixed_question = ''

    need_to_fix_numbers = [num for num in gpt_answer if num != answer]
    if len(need_to_fix_numbers) >= 1:
        fixed_numbers_str = ', '.join(map(str, need_to_fix_numbers))
        fixed_system_msg = {'role':'system', 'content': fixed_system_prompt}
        fixed_user_msg={'role':'user', 'content': fixed_user_prompt.format(question, fixed_numbers_str)}

        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages = [fixed_system_msg, fixed_user_msg]
        )

        result = response.choices[0].message.content
        question_lines = question.split('\n')
        for fixed_choice in result.split('\n'):
            fixed_choice_text = fixed_choice.strip()
                
            for i in range(1, len(question_lines)):
                if fixed_choice_text.startswith(str(i)):
                    question_lines[i] = fixed_choice_text

        fixed_question = '\n'.join(question_lines)

    if fixed_question:
        new_item = {'question':fixed_question, 'answer':answer}
    else:
        new_item = {'question':question, 'answer':answer}

    fixed_results.append(new_item)
            


100%|██████████| 623/623 [11:43<00:00,  1.13s/it]


# 데이터 전처리&저장

In [56]:
fixed_df = pd.DataFrame(fixed_results)
fixed_df



,question,answer
0,['디지털 뱅킹과 모바일 뱅킹 서비스의 발전에 따른 규제와 사이버 보안 문제를 고려...,1
1,"['신흥 시장에서 차용자의 신용 위험을 관리하기 위한 여러 전략 중에서, 정치적 및...",5
2,"['M-Pesa가 아프리카에서 금융 서비스 접근성을 제고하는 데 기여했지만, 규제와...",3
3,['PMJDY(Pradhan Mantri Jan Dhan Yojana)가 인도에서 ...,2
4,['DBS 은행과 JPMorgan Chase 은행의 디지털 이니셔티브가 상호작용 방...,5
...,...,...
618,['상업은행이 다양한 만기의 대출과 예금을 관리하면서 발생하는 금리 리스크를 최소화...,1
619,"['신용 포트폴리오 관리자가 대출자의 신용을 평가할 때, 어떤 요소가 신용 한도나 ...",1
620,['대출 신청자가 대출 승인을 받기 위해 제출해야 하는 정보가 신용 위험 평가 도구...,2
621,['VaR와 ES의 적용에서 발생할 수 있는 위험을 관리하기 위한 방법으로 가장 적...,5


In [44]:
import re

def preprocess_questions(question):
    question_lines = question.split('\n')

    number_to_eng = {
        '1. ': 'A. ',
        '2. ': 'B. ',
        '3. ': 'C. ',
        '4. ': 'D. ',
        '5. ': 'E. ',
    }

    new_question_lines = []
    for idx, line in enumerate(question_lines):
        new_line = ''
        for i in range(6):
            choice_string = f"{str(i)}. "
            if line.startswith(choice_string):
                new_line = re.sub(choice_string, number_to_eng[choice_string], line)
                break
        if new_line:
            new_question_lines.append(new_line)
        else:
            new_question_lines.append(line)
    
    question_text = "### 질문: " + new_question_lines[0]
    choice_text = "### 선택지:\n" + '\n'.join(new_question_lines[1:])
    result = question_text + "\n" + choice_text

    return result

def preprocess_answer(answer):
    answer = int(answer)
    
    number_to_eng = {
        1: 'A',
        2: 'B',
        3: 'C',
        4: 'D',
        5: 'E',
    }

    if 0 < answer < 6:
        result = number_to_eng[answer] 
    else:
        result = 0
    return result

In [45]:
fixed_df['question'] = fixed_df['question'].apply(preprocess_questions)
fixed_df['answer'] = fixed_df['answer'].apply(preprocess_answer)

In [46]:
def is_numeric(val):
    if val == 0:
        return False
    else:
        return True 


print(len(fixed_df))
fixed_df = fixed_df[fixed_df['answer'].apply(is_numeric)]
fixed_df = fixed_df.reset_index(drop=True)
print(len(fixed_df))

623
623


In [47]:
fixed_df.to_csv('/root/KRX_LLM/data/fixed_mcqa_data5.csv', index=False)